In [1]:
pip install tensorflow gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 MB 30.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.5/95.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 82.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 2.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
# AI 3D Model Generator Capstone Project

# Import Libraries
import json
import os
from scipy.io import loadmat
import numpy as np
import cv2
from scipy.ndimage import zoom
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Reshape, Input
from tensorflow.keras.utils import Sequence
import matplotlib.pyplot as plt
from skimage.measure import marching_cubes
from scipy.ndimage import gaussian_filter
import tempfile
import gradio as gr

# Load Pix3D Annotations
def load_pix3d_annotations(base_dir, json_filename='pix3d.json'):
    json_path = os.path.join(base_dir, json_filename)
    if not os.path.exists(json_path):
        raise FileNotFoundError(f"JSON file '{json_path}' not found.")
    with open(json_path, 'r') as f:
        pix3d_data = json.load(f)

    valid_samples = []
    for item in pix3d_data:
        if item['img'] and item['model'] and item.get('voxel'):
            valid_samples.append({
                'image_path': os.path.join(base_dir, item['img']),
                'voxel_path': os.path.join(base_dir, item['voxel']),
                'model_path': os.path.join(base_dir, item['model'])
            })

    print(f"Found {len(valid_samples)} valid samples.")
    return valid_samples

# Dataset Path
dataset_path = '/kaggle/input/pix3d-dataset'
samples = load_pix3d_annotations(dataset_path)

# Train Test Split
train_samples, val_samples = train_test_split(samples, test_size=0.2, random_state=42)

# Extract Point Cloud from Voxel
def extract_point_cloud_from_voxel(voxel_grid, threshold=0.5):
    points = np.argwhere(voxel_grid.squeeze() > threshold)
    return points

# Data Generator
class Pix3DPointCloudGenerator(Sequence):
    def __init__(self, samples, batch_size=8, image_size=64, num_points=2048, shuffle=True):
        self.samples = samples
        self.batch_size = batch_size
        self.image_size = image_size
        self.num_points = num_points
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.samples) / self.batch_size))

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.samples))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        batch_samples = [self.samples[k] for k in indexes]

        x, y = self.__data_generation(batch_samples)
        return np.array(x), np.array(y)

    def __data_generation(self, batch_samples):
        x = []
        y = []
        for sample in batch_samples:
            try:
                image = cv2.imread(sample['image_path'])
                if image is None:
                    continue
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, (self.image_size, self.image_size))
                image = image / 255.0

                voxel_data = loadmat(sample['voxel_path'])
                if 'voxel' in voxel_data:
                    voxel = voxel_data['voxel']
                elif 'model' in voxel_data:
                    voxel = voxel_data['model']
                else:
                    continue
                if voxel.ndim != 3:
                    continue
                voxel = voxel.astype('float32')

                scale_factor = 32 / voxel.shape[0]
                voxel = zoom(voxel, (scale_factor, scale_factor, scale_factor), order=0)

                points = extract_point_cloud_from_voxel(voxel)
                if len(points) >= self.num_points:
                    selected_idx = np.random.choice(len(points), self.num_points, replace=False)
                    points = points[selected_idx]
                else:
                    pad_size = self.num_points - len(points)
                    pad_points = np.zeros((pad_size, 3))
                    points = np.vstack((points, pad_points))

                x.append(image)
                y.append(points)

            except Exception as e:
                print(f"Error loading {sample['voxel_path']} -> {e}")
                continue

        return x, y

# Build PointCloudNet Model
def pointcloudnet_build(image_size=64, num_points=2048):
    model = Sequential()
    model.add(Input(shape=(image_size, image_size, 3)))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))
    model.add(Flatten())
    model.add(Dense(num_points * 3, activation='linear'))
    model.add(Reshape((num_points, 3)))

    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

# Build Generators
train_generator = Pix3DPointCloudGenerator(train_samples, batch_size=8, image_size=64, num_points=2048)
val_generator = Pix3DPointCloudGenerator(val_samples, batch_size=8, image_size=64, num_points=2048)

# Build and Train Model
model = pointcloudnet_build()
model.fit(train_generator, validation_data=val_generator, epochs=20)
model.save('pointcloudnet_model.h5')

print("Model Training completed and saved.")

# Mesh Generation from Point Cloud
def generate_mesh_from_pointcloud(points, grid_size=64):
    voxel_grid = np.zeros((grid_size, grid_size, grid_size))
    for point in points:
        x, y, z = np.clip(point.astype(int), 0, grid_size - 1)
        voxel_grid[x, y, z] = 1.0

    voxel_grid = gaussian_filter(voxel_grid, sigma=1)
    verts, faces, normals, _ = marching_cubes(voxel_grid, level=0.5)
    return verts, faces

# Export Mesh to OBJ
def export_mesh_as_obj(vertices, faces, obj_filename='generated_mesh.obj'):
    with open(obj_filename, 'w') as f:
        for v in vertices:
            f.write(f"v {v[0]} {v[1]} {v[2]}\n")
        for face in faces:
            f.write(f"f {int(face[0])+1} {int(face[1])+1} {int(face[2])+1}\n")
    print(f"Exported mesh to {obj_filename}")

# Gradio Prediction Function
def predict_mesh(image_input):
    try:
        image = cv2.resize(image_input, (64, 64))
        image = image / 255.0
        image = np.expand_dims(image, axis=0)

        point_cloud = model.predict(image)[0]
        verts, faces = generate_mesh_from_pointcloud(point_cloud)

        temp_dir = tempfile.mkdtemp()
        obj_path = os.path.join(temp_dir, 'generated_mesh.obj')
        export_mesh_as_obj(verts, faces, obj_filename=obj_path)

        fig = plt.figure(figsize=(8, 8))
        ax = fig.add_subplot(111, projection='3d')
        ax.plot_trisurf(verts[:, 0], verts[:, 1], faces, verts[:, 2], color='lightblue', edgecolor='gray')
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        plt.tight_layout()
        plt.close(fig)

        return fig, obj_path

    except Exception as e:
        print(f"Error: {e}")
        return None, None

# Gradio Interface
iface = gr.Interface(
    fn=predict_mesh,
    inputs=gr.Image(type='numpy', label="Upload 2D Image"),
    outputs=[
        gr.Plot(label="Predicted 3D Mesh Visualization"),
        gr.File(label="Download 3D .obj Model")
    ],
    title="AI 3D Model Generator",
    description="Upload an image to convert to a 3D mesh model and export as .obj",
    live=True
)

iface.launch()


2025-07-02 05:33:09.679595: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751434390.044541      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751434390.147232      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Found 10069 valid samples.


I0000 00:00:1751434413.955316      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1751434413.956184      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20


I0000 00:00:1751434420.102820     109 service.cc:148] XLA service 0x7cb0c40103f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1751434420.104447     109 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1751434420.104469     109 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1751434420.464296     109 cuda_dnn.cc:529] Loaded cuDNN version 90300


...

libpng warning: iCCP: known incorrect sRGB profile


 472/1006 ━━━━━━━━━━━━━━━━━━━━ 1:52 211ms/step - loss: 28.0992 - mae: 3.7598

libpng warning: iCCP: known incorrect sRGB profile


1006/1006 ━━━━━━━━━━━━━━━━━━━━ 266s 265ms/step - loss: 28.1211 - mae: 3.7683 - val_loss: 83.6998 - val_mae: 7.2261
Model Training completed and saved.
* Running on local URL:  http://127.0.0.1:7860
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://34a5aeecf17e5fcd0b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 667ms/step
Exported mesh to /tmp/tmpt38a5uay/generated_mesh.obj
Error: OpenCV(4.11.0) /io/opencv/modules/imgproc/src/resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Exported mesh to /tmp/tmpn7krwhgv/generated_mesh.obj
